<a href="https://colab.research.google.com/github/Madhavsingla1/Capstone_Project/blob/main/TF_IDF_Bag_of_words_and_Word2Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from scipy import *
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy
from scipy import spatial
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
STOP_WORDS = stopwords.words("english")
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn.metrics as classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import precision_recall_curve, auc, roc_curve ,f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df= pd.read_csv("/content/drive/MyDrive/Capstone Data/train.csv") # importing the dataset

df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df.describe()

,id,qid1,qid2,is_duplicate
count,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198
std,116708.614503,157751.700002,159903.182629,0.482588
min,0.000000,1.000000,2.000000,0.000000
25%,101072.250000,74437.500000,74727.000000,0.000000
50%,202144.500000,192182.000000,197052.000000,0.000000
75%,303216.750000,346573.500000,354692.500000,1.000000
max,404289.000000,537932.000000,537933.000000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404290 non-null  object
 4   question2     404290 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


# **TF-IDF**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
documentA = 'the man went out for a walk'
documentB = 'the children sat around the fire'

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df7 = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
print(df7.head())

     around  children      fire      for      man      out       sat  \
0  0.000000  0.000000  0.000000  0.42616  0.42616  0.42616  0.000000   
1  0.407401  0.407401  0.407401  0.00000  0.00000  0.00000  0.407401   

        the     walk     went  
0  0.303216  0.42616  0.42616  
1  0.579739  0.00000  0.00000  


In [ ]:
print(vectors)

  (0, 8)	0.42615959880289433
  (0, 3)	0.42615959880289433
  (0, 5)	0.42615959880289433
  (0, 9)	0.42615959880289433
  (0, 4)	0.42615959880289433
  (0, 7)	0.3032160644503863
  (1, 2)	0.40740123733358447
  (1, 0)	0.40740123733358447
  (1, 6)	0.40740123733358447
  (1, 1)	0.40740123733358447
  (1, 7)	0.5797386715376657


In [ ]:
print(feature_names)

['around', 'children', 'fire', 'for', 'man', 'out', 'sat', 'the', 'walk', 'went']


In [ ]:
df['question1']

0         What is the step by step guide to invest in sh...
1         What is the story of Kohinoor (Koh-i-Noor) Dia...
2         How can I increase the speed of my internet co...
3         Why am I mentally very lonely? How can I solve...
4         Which one dissolve in water quikly sugar, salt...
                                ...                        
404285    How many keywords are there in the Racket prog...
404286            Do you believe there is life after death?
404287                                    What is one coin?
404288    What is the approx annual cost of living while...
404289                What is like to have sex with cousin?
Name: question1, Length: 404290, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
question = list(df['question1']) + list(df['question2'])

questions = [ques for ques in question if ques not in STOP_WORDS]

tfidf = TfidfVectorizer(lowercase=False, )
x=tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
print(x) # first id , feautre_names no. , tfidf score

  (0, 77824)	0.28810089872803335
  (0, 83151)	0.27283405422911383
  (0, 97832)	0.2994105898351769
  (0, 77574)	0.20122105586038397
  (0, 78968)	0.2870081522008514
  (0, 103601)	0.09954570141748624
  (0, 74674)	0.3612041095478802
  (0, 60060)	0.19437825565580152
  (0, 100415)	0.6561773827369753
  (0, 103025)	0.07891022032825844
  (0, 79172)	0.0938889428826086
  (0, 51739)	0.07923245022121653
  (1, 16010)	0.44275849179086657
  (1, 34922)	0.497188073608617
  (1, 27864)	0.4741291764130098
  (1, 27867)	0.4632308754936714
  (1, 87457)	0.10928916808876182
  (1, 100638)	0.2895425035439381
  (1, 103025)	0.07944306353728867
  (1, 79172)	0.09452293028512647
  (1, 51739)	0.07976746929554651
  (2, 50118)	0.44691071060685306
  (2, 106157)	0.29128739079040816
  (2, 107880)	0.30756056894249173
  (2, 63504)	0.42337596422607426
  :	:
  (808573, 92091)	0.21004595937718346
  (808573, 59992)	0.16564744818997473
  (808573, 91680)	0.23467372994318145
  (808573, 74907)	0.19931796385379638
  (808573, 92159)	0.

In [ ]:
tfidf.idf_

array([10.04915306,  7.30458485, 13.90988277, ..., 13.50441767,
       13.50441767, 13.50441767])

In [ ]:
abf = tfidf.get_feature_names()
abf[10000:10020]

['BookRenter',
 'BookSellBuy',
 'Booker',
 'Booking',
 'BookinginBit',
 'Bookit',
 'Booklet',
 'Bookmarked',
 'Bookmarks',
 'Bookmyoffer',
 'Books',
 'Bookshelf',
 'Bookshelves',
 'Booktract',
 'Bool',
 'Boolean',
 'Boom',
 'Boomer',
 'Boomerang',
 'Boomers']

In [ ]:
word2tfidf['what']

4.767607133196319

In [ ]:
len(abf)==len(word2tfidf)

True

In [ ]:
d = documentA.split(' ')
print(d)

l = [word for word in d if word not in STOP_WORDS]
print(l)

['the', 'man', 'went', 'out', 'for', 'a', 'walk']
['man', 'went', 'walk']


In [ ]:
for i in l:
  print("{0}={1}".format(i,word2tfidf[i]))

man=6.6814943226847685
went=8.40455123832601
walk=8.692233310777791


## Xgboost

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect.transform(df['question1'].values)
trainq2_trans = tfidf_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

xgb_model = xgb.XGBClassifier(n_estimators=80,random_state=42).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)
#, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
print(accuracy_score(y_valid,xgb_prediction))
print(classification_report(y_valid, xgb_prediction))

## Random Forest

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = tfidf_vect.transform(df['question1'].values)
trainq2_trans = tfidf_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

clf = RandomForestClassifier(n_estimators=80).fit(X_train,y_train)
adb = clf.predict(X_valid)
print('word level tf-idf validation score:', f1_score(y_valid, clf.predict(X_valid), average='macro'))

word level tf-idf validation score: 0.769154332846925


In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
print(accuracy_score(y_valid,adb))
print(classification_report(y_valid,adb))

0.8009983810037776
              precision    recall  f1-score   support

           0       0.79      0.93      0.85     83975
           1       0.83      0.58      0.68     49441

    accuracy                           0.80    133416
   macro avg       0.81      0.76      0.77    133416
weighted avg       0.81      0.80      0.79    133416



## AdaBoost

In [ ]:
Ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=80).fit(X_train, y_train) 
Ada_prediction = Ada_model.predict(X_valid)

In [ ]:
print(accuracy_score(y_valid,Ada_prediction))
print(classification_report(y_valid,Ada_prediction))

0.7239911255021887
              precision    recall  f1-score   support

           0       0.79      0.76      0.78     83975
           1       0.62      0.67      0.64     49441

    accuracy                           0.72    133416
   macro avg       0.71      0.71      0.71    133416
weighted avg       0.73      0.72      0.73    133416



# **Word2vector**

In [ ]:
import gensim
from gensim.models import Word2Vec
import multiprocessing


csv_path = "/content/drive/MyDrive/Extra data Capstone/Hotel_Reviews.csv" #random dataset

df1 = pd.read_csv(csv_path)

In [ ]:
df1.head()

In [ ]:
df1.Description

In [ ]:
def remover(text):  #function for removing the Stop words
  text_t = text.split(" ")  
  l2 = [words.lower() for words in text_t if words not in STOP_WORDS]
  return l2;

In [ ]:
Description = df1.Description.apply(remover)
Description2 = df1.Description.apply(gensim.utils.simple_preprocess)
print(Description[0])
print(Description2[0])

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
model_sample = Word2Vec(window=5,min_count=4,workers = cores-1)
print(model_sample)

In [ ]:
model_sample.build_vocab(Description,progress_per=1000)

In [ ]:
print(model_sample)
model_sample.epochs

In [ ]:
model_sample.train(Description,total_examples=model_sample.corpus_count,epochs=10)

In [ ]:
model_sample.wv.most_similar("bad")

### Main DataSet

In [ ]:
question = list(df['question1'])

Using Stop_words list for preprocessing

In [ ]:
def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text

In [ ]:
questions= []
for ques in question:
  ques = clean_data(ques)
  questions.append(remover(ques))


In [ ]:
print(questions[1:10])

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
model1 = Word2Vec(window=5,min_count=4,workers = cores-1)
print(model1)

In [ ]:
model1.build_vocab(questions,progress_per=10000)
print(model1)
model1.epochs

In [ ]:
model1.train(questions,total_examples=model1.corpus_count,epochs=5)

In [ ]:
model1.wv.most_similar("bad")

In [ ]:
model1.wv.similarity(w1="bad",w2="good")

In [ ]:
model1.wv.similarity(w1="bad",w2="bad")

In [ ]:
model1.wv.similarity(w1="bad",w2="horrible")

### Using in Built Preprocesser

In [ ]:
question2=[]
for ques in question:
  question2.append(gensim.utils.simple_preprocess(ques))

In [ ]:
print(question2[1:10])

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
model2 = Word2Vec(window=5,min_count=4,workers = cores-1)
print(model2)

In [ ]:
model2.build_vocab(question2,progress_per=10000)
print(model2)
model2.epochs

In [ ]:
model2.train(question2,total_examples=model2.corpus_count,epochs=5)

In [ ]:
model2.wv.most_similar("bad")

In [ ]:
model2.wv.similarity(w1="bad",w2="good")

In [ ]:
model2.wv.similarity(w1="bad",w2="bad")

### **Using Skip-Gram**

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)
model_sg = Word2Vec(window=5,min_count=4,workers = cores-1,sg=3)
print(model_sg)

In [ ]:
model_sg.build_vocab(questions,progress_per=10000)
print(model_sg)
model_sg.epochs

In [ ]:
model_sg.train(questions,total_examples=model_sg.corpus_count,epochs=4)

In [ ]:
model_sg.wv.most_similar("bad")

In [ ]:
print(model_sg)

In [ ]:
model_sg.wv.most_similar("good")

### Checking Similiarty

In [ ]:
def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [ ]:
index2word_set = set(model_sg.wv.index2word) 
s1_afv = avg_feature_vector(df['question1'][0], model=model_sg, num_features=100, index2word_set=index2word_set)
s2_afv = avg_feature_vector(df['question2'][0], model=model_sg, num_features=100, index2word_set=index2word_set)

In [ ]:
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)

In [ ]:
data = {}
s1_afv = avg_feature_vector(df['question1'][0], model=model_sg, num_features=100, index2word_set=index2word_set)
for j in df['question2']:
  s2_afv = avg_feature_vector(j, model=model_sg, num_features=100, index2word_set=index2word_set)
  sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
  data[sim]=j

In [ ]:
data

# Bag of Word

In [ ]:
q1=df['question1']
q2=df['question2']
ques = np.concatenate((q1,q2),axis=0)
print(len(ques))

In [ ]:
vectorizer2= CountVectorizer()
vectorizer2.fit(ques)
print(len(vectorizer2.vocabulary_))

In [ ]:
bow_vector = vectorizer2.transform(ques)
print(bow_vector.shape)
print(bow_vector[0])

## Xgboost

In [ ]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = count_vect.transform(df['question1'].values)
trainq2_trans = count_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

xgb_model = xgb.XGBClassifier(n_estimators=80).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)

from sklearn.metrics import f1_score, classification_report, accuracy_score

print('training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
print(classification_report(y_valid, xgb_prediction))

training score: 0.5583329839621789
validation score: 0.5573457026584304
              precision    recall  f1-score   support

           0       0.67      0.96      0.79     83975
           1       0.77      0.20      0.32     49441

    accuracy                           0.68    133416
   macro avg       0.72      0.58      0.56    133416
weighted avg       0.71      0.68      0.62    133416



In [ ]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(pd.concat((df['question1'],df['question2'])).unique())
trainq1_trans = count_vect.transform(df['question1'].values)
trainq2_trans = count_vect.transform(df['question2'].values)
labels = df['is_duplicate'].values
X = scipy.sparse.hstack((trainq1_trans,trainq2_trans))
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.33, random_state = 42)

clf = RandomForestClassifier(n_estimators=80).fit(X_train,y_train)
adb = clf.predict(X_valid)
print('word level tf-idf validation score:', f1_score(y_valid, clf.predict(X_valid), average='macro'))

word level tf-idf validation score: 0.7822300988561282


In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
print(accuracy_score(y_valid,adb))
print(classification_report(y_valid,adb))

0.8103300953408886
              precision    recall  f1-score   support

           0       0.80      0.93      0.86     83975
           1       0.83      0.61      0.70     49441

    accuracy                           0.81    133416
   macro avg       0.82      0.77      0.78    133416
weighted avg       0.81      0.81      0.80    133416



In [ ]:
Ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=80).fit(X_train, y_train) 
Ada_prediction = Ada_model.predict(X_valid)

In [ ]:
print(accuracy_score(y_valid,Ada_prediction))
print(classification_report(y_valid,Ada_prediction))

0.7627271091923008
              precision    recall  f1-score   support

           0       0.76      0.92      0.83     83975
           1       0.79      0.49      0.61     49441

    accuracy                           0.76    133416
   macro avg       0.77      0.71      0.72    133416
weighted avg       0.77      0.76      0.75    133416

